In [1]:
#Initializing an LLM and Setting up Langsmith

import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Langchain Tutorial"

from langchain_groq import ChatGroq

llm = ChatGroq(model="openai/gpt-oss-20b")

## Introduction to Tools
Tools are a way augment our LLMs with code execution. A tool is simply a function formatted so that our agent can undertstand how to use it, and then execute it. Let's start by creating a few simple tools.

We can use the @tool decorator to create an LLM-compatible tool from a standard python function — this function should include a few things for optimal performance:

* A docstring describing what the tool does and when it should be used, this will be read by our LLM/agent and used to decide when to use the tool, and also how to use the tool.

* Clear parameter names that ideally tell the LLM what each parameter is, if it isn't clear we make sure the docstring explains what the parameter is for and how to use it.

* Both parameter and return type annotations.

In [2]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

With the @tool decorator our function is turned into a StructuredTool object, which we can see below:

In [3]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x000002596B9D9BC0>)

In [4]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [5]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to the below:

In [6]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

This is then passed into the tool function as kwargs (keyword arguments) as indicated by the ** operator - the ** operator is used to unpack the dictionary into keyword arguments.

In [7]:
exponentiate.func(**llm_output_dict)

25

### Creating an Agent

We're going to construct a simple tool calling agent. We will use LangChain Epression Language (LCEL) to construct the agent. We will cover LCEL more in the next chapter, but for now - all we need to know is that our agent will be constructed using syntax and components like so:

agent = (
    <input parameters, including chat history and user query>
    | <prompt>
    | <LLM with tools>
)
We need this agent to remember previous interactions within the conversation. To do that, we will use the ChatPromptTemplate with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent will write it's "notes" as it is working through multiple internal thought and tool-use steps to produce a final output to the user.

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a Helpful Assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

When creating an agent we need to add conversational memory to make the agent remember previous interactions. We'll be using the older ConversationBufferMemory class rather than the newer RunnableWithMessageHistory — the reason being that we will also be using the older create_tool_calling_agent and AgentExecutor method and class.

In the 05 chapter we will be using the newer RunnableWithMessageHistory class as we'll be building a custom AgentExecutor.

In [9]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key = "chat_history",
    return_messages= True
)

C:\Users\HP\AppData\Local\Temp\ipykernel_3680\800012107.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Now we will initialize our agent. For that we need:

* llm: as already defined <br>
* tools: to be defined (just a list of our previously defined tools) <br>
* prompt: as already defined <br>
* memory: as already defined <br>

In [10]:
from langchain.agents import create_tool_calling_agent

tools = [add, subtract, multiply, exponentiate]

agent = create_tool_calling_agent(
    llm = llm,
    tools = tools,
    prompt = prompt
)

Our agent by itself is like one-step of our agent execution loop. So, if we call the agent.invoke method it will get the LLM to generate a single response and go no further, so no tools will be executed, and no next iterations will be performed.

We can see this by asking a query that should trigger a tool call:

In [13]:
res = agent.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps": []  # agent will append it's internal steps here
})

In [17]:
from pprint import pprint
pprint(res)

[ToolAgentAction(tool='multiply', tool_input={'x': 10.7, 'y': 7.68}, log="\nInvoking: `multiply` with `{'x': 10.7, 'y': 7.68}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to calculate 10.7 * 7.68. Use function multiply.', 'tool_calls': [{'id': 'fc_2b78c3ba-99e5-4986-9cdb-053b3db77fb6', 'function': {'arguments': '{"x":10.7,"y":7.68}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 229, 'total_tokens': 278, 'completion_time': 0.064713115, 'prompt_time': 0.015573745, 'queue_time': 0.246054624, 'total_time': 0.08028686}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_77f8660d1d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a5395bf7-75b1-4d74-a227-ddfe2c458e8a-0', tool_calls=[{'name': 'multiply', 'args': {'x': 10.7, 'y': 7.68}, 'id': 'fc_2b78c3ba-99e5-4986-9cdb-053b3db77fb6', 'type': 'tool_call'}], usage_metadata

Here, we can see the LLM has generated that we should use the multiply tool and the tool input should be {"x": 10.7, "y": 7.68}. However, the tool is not executed. For that to happen we need an agent execution loop, which will handle the multiple iterations of generation to tool calling to generation, etc.

We use the AgentExecutor class to handle the execution loop:

In [19]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    memory = memory, 
    verbose = True
)

Now let's try the same query with the executor, note that the intermediate_steps parameter that we added before is no longer needed as the executor handles it internally.

In [20]:
agent_executor.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 10.7, 'y': 7.68}`


82.1759999999999910.7 × 7.68 = **82.176** (rounded to three decimal places).

> Finished chain.


{'input': 'what is 10.7 multiplied by 7.68?',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7\u202f×\u202f7.68\u202f=\u202f**82.176** (rounded to three decimal places).', additional_kwargs={}, response_metadata={})],
 'output': '10.7\u202f×\u202f7.68\u202f=\u202f**82.176** (rounded to three decimal places).'}

In [21]:
10.7*7.68

82.17599999999999

Let's test our agent with some memory and tool use. First, we tell it our name, then we will perform a few tool calls, then see if the agent can still recall our name.

First, give the agent our name:

In [25]:
agent_executor.invoke({
    "input": "My name is Ankur",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
Got it—your name is **Ankur**. Nice to meet you, Ankur! How can I help you today?

> Finished chain.


{'input': 'My name is Ankur',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7\u202f×\u202f7.68\u202f=\u202f**82.176** (rounded to three decimal places).', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, James! How can I help you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Let’s break it down step by step:\n\n1. **Multiplication first** (according to order of operations):  \n   \\[\n   4 \\times 2 = 8\n   \\]\n\n2. **Addition and subtraction next**:  \n   \\[\n   9 + 10 - 8 = 19 - 8 = 11\n   \\]\n\n3. **Raise the result to the power of 3**:  \n   \\[\n   11^3 = 11 \\times 11 \\times 11 = 1331\n   \\]\n\nSo, \\(\\boxe

In [26]:
agent_executor.invoke({
    "input": "What is nine plus 10, minus 4 * 2, to the power of 3",
    "chat_history": memory
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 4, 'y': 2}`


8.0Let’s evaluate it step‑by‑step:

1. **Multiplication first**  
   \[
   4 \times 2 = 8
   \]

2. **Addition and subtraction next**  
   \[
   9 + 10 - 8 = 19 - 8 = 11
   \]

3. **Raise the result to the 3rd power**  
   \[
   11^3 = 11 \times 11 \times 11 = 1331
   \]

So the value is **1331**.

> Finished chain.


{'input': 'What is nine plus 10, minus 4 * 2, to the power of 3',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7\u202f×\u202f7.68\u202f=\u202f**82.176** (rounded to three decimal places).', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, James! How can I help you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Let’s break it down step by step:\n\n1. **Multiplication first** (according to order of operations):  \n   \\[\n   4 \\times 2 = 8\n   \\]\n\n2. **Addition and subtraction next**:  \n   \\[\n   9 + 10 - 8 = 19 - 8 = 11\n   \\]\n\n3. **Raise the result to the power of 3**:  \n   \\[\n   11^3 = 11 \\times 11 \\time

In [27]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
Your name is **Ankur**.

> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7\u202f×\u202f7.68\u202f=\u202f**82.176** (rounded to three decimal places).', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, James! How can I help you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Let’s break it down step by step:\n\n1. **Multiplication first** (according to order of operations):  \n   \\[\n   4 \\times 2 = 8\n   \\]\n\n2. **Addition and subtraction next**:  \n   \\[\n   9 + 10 - 8 = 19 - 8 = 11\n   \\]\n\n3. **Raise the result to the power of 3**:  \n   \\[\n   11^3 = 11 \\times 11 \\times 11 = 1331\n   \\]\n\nSo, \\(\\boxed

### SerpAPI Weather Agent
In this example, we'll be using the same agent and executor setup as before, but we'll be adding the SerpAPI service to allow our agent to search the web for information.

To use this tool, you need an API key, with the free plan you can use up to 100 searches per month.